In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
df_sales = pd.read_csv("./data/df_sales_clean.csv")

In [3]:
df_prices = pd.read_csv("./data/df_price_clean.csv")

In [5]:
df_calendar_with_events = pd.read_csv("./data/daily_calendar_with_events.csv")

### Rellenamos los nulos de Calendar_with_events por 0 y en los que haya evento los reemplazamos por 1 

In [6]:
df_calendar_with_events["event"].fillna(0, inplace=True)

C:\Users\rafas\AppData\Local\Temp\ipykernel_5760\3695517197.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_calendar_with_events["event"].fillna(0, inplace=True)


In [7]:
df_calendar_with_events["event"] = df_calendar_with_events["event"].apply(lambda x: 1 if x != 0 else 0)

In [8]:
df_calendar_with_events["date"]= pd.to_datetime(df_calendar_with_events['date'], format='%Y-%m-%d') # change type to datetime

In [9]:
del(df_sales["Unnamed: 0"])
del(df_prices["Unnamed: 0"])


In [10]:
df_sales["date"] = pd.to_datetime(df_sales["date"])
df_calendar["date"] = pd.to_datetime(df_calendar["date"])

In [11]:
df_sales.sample(2)

,id,store,date,sales
6024956,ACCESORIES_1_137_BOS_3,Back_Bay,2011-08-14,0
41989133,HOME_&_GARDEN_2_259_NYC_2,Harlem,2014-11-06,0


In [12]:
df_calendar.sample(2)

,date,weekday,d,event
374,2012-02-07,Tuesday,d_375,no event
866,2013-06-13,Thursday,d_867,no event


In [13]:
df_prices.sample(2)

,item,category,store_code,yearweek,sell_price
3318833,SUPERMARKET_3_690,SUPERMARKET,BOS_1,201519.0,11.976
4497477,SUPERMARKET_2_223,SUPERMARKET,BOS_3,201331.0,2.616


In [14]:
def create_yearweek_df(start_date_str, end_date_str, initial_week):
    """
    Create dataframe with week and dates
    """
    #Convertir las cadenas de fecha a objetos datetime
    start_date = pd.to_datetime(start_date_str)
    end_date = pd.to_datetime(end_date_str)

    # Generar el rango de fechas
    date_range = pd.date_range(start_date, end_date)
    df = pd.DataFrame(date_range, columns=['date'])

    # Calcular el número de días desde la fecha de inicio
    df['dias_desde_inicio'] = (df['date'] - start_date).dt.days

    # Calcular el número de la semana ajustado desde la fecha inicial
    df['week_number'] = ((df['dias_desde_inicio'] // 7) + initial_week).astype(int)

    # Crear una columna 'yearweek' que combine el año y la semana
    df['year'] = df['date'].dt.year
    df['yearweek'] = df['year'].astype(str) + df['week_number'].apply(lambda x: f"{x:02d}")

    # Eliminar las columnas auxiliares
    df.drop(['dias_desde_inicio', 'week_number', 'year'], axis=1, inplace=True)

    return df

    # Crear DataFrames para cada año
df_2011 = create_yearweek_df('2011-01-29', '2011-12-30', 5)
df_2012_1 = create_yearweek_df('2011-12-31', '2012-01-01', 0)
df_2012 = create_yearweek_df('2012-01-02', '2012-12-30', 1)
df_2012_53 = pd.DataFrame([['2012-12-31','201253']],columns=['date','yearweek'])
df_2013 = create_yearweek_df('2013-01-01', '2013-12-31', 0)
df_2014 = create_yearweek_df('2014-01-01', '2014-12-31', 0)
df_2015 = create_yearweek_df('2015-01-01', '2015-12-31', 0)
df_2016_1 = create_yearweek_df('2016-01-01', '2016-01-02', 0)
df_2016 = create_yearweek_df('2016-01-03', '2016-04-24', 1)

df_2012_1.loc[0,'yearweek']='201152'

all_years_df = pd.concat([df_2011, df_2012_1, df_2012, df_2012_53, df_2013, df_2014, df_2015,df_2016_1, df_2016]).reset_index(drop=True)

In [15]:
all_years_df['date']= pd.to_datetime(all_years_df['date'], format='%Y-%m-%d') # change type to datetime
df_prices['yearweek'] = df_prices['yearweek'].astype(str).apply(lambda x: x[:-2]) # delete last two characters

In [16]:
df_prices.head(2)

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329,12.7414


In [17]:
df_calendar.head(1)

,date,weekday,d,event
0,2011-01-29,Saturday,d_1,no event


In [18]:
all_years_df_merged = pd.merge(all_years_df, df_calendar_with_events, on='date', how='left')

In [19]:
all_years_df_merged.drop(columns=["weekday_int","d"], axis=1, inplace=True)

In [20]:
all_years_df_merged

,date,yearweek,weekday,event
0,2011-01-29,201105,Saturday,0
1,2011-01-30,201105,Sunday,0
2,2011-01-31,201105,Monday,0
3,2011-02-01,201105,Tuesday,0
4,2011-02-02,201105,Wednesday,0
...,...,...,...,...
1908,2016-04-20,201616,Wednesday,0
1909,2016-04-21,201616,Thursday,0
1910,2016-04-22,201616,Friday,0
1911,2016-04-23,201616,Saturday,0


In [21]:
del(df_prices["category"])

In [22]:
df_prices

,item,store_code,yearweek,sell_price
0,ACCESORIES_1_001,NYC_1,201328,12.7414
1,ACCESORIES_1_001,NYC_1,201329,12.7414
2,ACCESORIES_1_001,NYC_1,201330,10.9858
3,ACCESORIES_1_001,NYC_1,201331,10.9858
4,ACCESORIES_1_001,NYC_1,201332,10.9858
...,...,...,...,...
6721781,SUPERMARKET_3_827,PHI_3,201613,1.2000
6721782,SUPERMARKET_3_827,PHI_3,201614,1.2000
6721783,SUPERMARKET_3_827,PHI_3,201615,1.2000
6721784,SUPERMARKET_3_827,PHI_3,201616,1.2000


In [23]:
df_merge = df_prices.merge(all_years_df_merged, on=['yearweek'], how='left') # merge df_prices with list of dates

In [24]:
df_merge["date"] = pd.to_datetime(df_merge["date"])

In [25]:
df_merge["id"] = df_merge["item"] + "_" + df_merge["store_code"]

In [26]:
df_58 = df_sales.merge(df_merge, on=["id", "date"], how="left")

In [27]:
df_58.dropna(subset=["sell_price"], inplace=True)

In [28]:
df_58.head(2)

,id,store,date,sales,item,store_code,yearweek,sell_price,weekday,event
7,ACCESORIES_1_008_NYC_1,Greenwich_Village,2011-01-29,12,ACCESORIES_1_008,NYC_1,201105,0.6118,Saturday,0.0
8,ACCESORIES_1_009_NYC_1,Greenwich_Village,2011-01-29,2,ACCESORIES_1_009,NYC_1,201105,2.0748,Saturday,0.0


In [29]:
df_58["income"] = df_58["sales"]*df_58["sell_price"]

In [30]:
df_58

,id,store,date,sales,item,store_code,yearweek,sell_price,weekday,event,income
7,ACCESORIES_1_008_NYC_1,Greenwich_Village,2011-01-29,12,ACCESORIES_1_008,NYC_1,201105,0.6118,Saturday,0.0,7.3416
8,ACCESORIES_1_009_NYC_1,Greenwich_Village,2011-01-29,2,ACCESORIES_1_009,NYC_1,201105,2.0748,Saturday,0.0,4.1496
9,ACCESORIES_1_010_NYC_1,Greenwich_Village,2011-01-29,0,ACCESORIES_1_010,NYC_1,201105,4.2161,Saturday,0.0,0.0000
11,ACCESORIES_1_012_NYC_1,Greenwich_Village,2011-01-29,0,ACCESORIES_1_012,NYC_1,201105,7.9534,Saturday,0.0,0.0000
14,ACCESORIES_1_015_NYC_1,Greenwich_Village,2011-01-29,4,ACCESORIES_1_015,NYC_1,201105,0.9310,Saturday,0.0,3.7240
...,...,...,...,...,...,...,...,...,...,...,...
58327365,SUPERMARKET_3_823_PHI_3,Queen_Village,2016-04-24,1,SUPERMARKET_3_823,PHI_3,201617,3.5760,Sunday,0.0,3.5760
58327366,SUPERMARKET_3_824_PHI_3,Queen_Village,2016-04-24,0,SUPERMARKET_3_824,PHI_3,201617,2.9760,Sunday,0.0,0.0000
58327367,SUPERMARKET_3_825_PHI_3,Queen_Village,2016-04-24,0,SUPERMARKET_3_825,PHI_3,201617,4.7760,Sunday,0.0,0.0000
58327368,SUPERMARKET_3_826_PHI_3,Queen_Village,2016-04-24,3,SUPERMARKET_3_826,PHI_3,201617,1.5360,Sunday,0.0,4.6080


In [31]:
df_calendar

,date,weekday,d,event
0,2011-01-29,Saturday,d_1,no event
1,2011-01-30,Sunday,d_2,no event
2,2011-01-31,Monday,d_3,no event
3,2011-02-01,Tuesday,d_4,no event
4,2011-02-02,Wednesday,d_5,no event
...,...,...,...,...
1908,2016-04-20,Wednesday,d_1909,no event
1909,2016-04-21,Thursday,d_1910,no event
1910,2016-04-22,Friday,d_1911,no event
1911,2016-04-23,Saturday,d_1912,no event


In [40]:
df_58.drop(columns=["store_code","yearweek"], inplace=True)

### Top 100 de productos que generan mayor numero de ingresos totales entre nuestras 10 tiendas

In [33]:
top100= df_58.groupby( ["item"])["income"].sum().sort_values(ascending=False).head(100).reset_index()

In [34]:
top100

,item,income
0,SUPERMARKET_3_586,1.754179e+06
1,SUPERMARKET_3_120,1.690645e+06
2,SUPERMARKET_3_090,1.623844e+06
3,SUPERMARKET_3_202,1.501405e+06
4,SUPERMARKET_3_587,1.171340e+06
...,...,...
95,SUPERMARKET_3_389,3.248685e+05
96,SUPERMARKET_3_541,3.231147e+05
97,SUPERMARKET_3_681,3.180257e+05
98,HOME_&_GARDEN_1_497,3.177413e+05


### Los guardamos en una lista

In [35]:
Lista_top100=list(top100["item"])

### Filtramos el df con las 58m de filas con la lista que hemos creado.

In [36]:
df_filtrado = df_58[df_58['item'].isin(Lista_top100)]

In [37]:
df_filtrado

,id,store,date,sales,item,store_code,yearweek,sell_price,weekday,event,income
103,ACCESORIES_1_108_NYC_1,Greenwich_Village,2011-01-29,4,ACCESORIES_1_108,NYC_1,201105,16.1196,Saturday,0.0,64.4784
591,HOME_&_GARDEN_1_027_NYC_1,Greenwich_Village,2011-01-29,4,HOME_&_GARDEN_1_027,NYC_1,201105,6.5500,Saturday,0.0,26.2000
615,HOME_&_GARDEN_1_053_NYC_1,Greenwich_Village,2011-01-29,0,HOME_&_GARDEN_1_053,NYC_1,201105,18.7125,Saturday,0.0,0.0000
700,HOME_&_GARDEN_1_140_NYC_1,Greenwich_Village,2011-01-29,4,HOME_&_GARDEN_1_140,NYC_1,201105,18.7500,Saturday,0.0,75.0000
737,HOME_&_GARDEN_1_177_NYC_1,Greenwich_Village,2011-01-29,0,HOME_&_GARDEN_1_177,NYC_1,201105,9.9625,Saturday,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
58327325,SUPERMARKET_3_783_PHI_3,Queen_Village,2016-04-24,0,SUPERMARKET_3_783,PHI_3,201617,6.5280,Sunday,0.0,0.0000
58327327,SUPERMARKET_3_785_PHI_3,Queen_Village,2016-04-24,16,SUPERMARKET_3_785,PHI_3,201617,3.6000,Sunday,0.0,57.6000
58327346,SUPERMARKET_3_804_PHI_3,Queen_Village,2016-04-24,2,SUPERMARKET_3_804,PHI_3,201617,2.2560,Sunday,0.0,4.5120
58327350,SUPERMARKET_3_808_PHI_3,Queen_Village,2016-04-24,0,SUPERMARKET_3_808,PHI_3,201617,1.1280,Sunday,0.0,0.0000


In [38]:
file_name = "Top100_prediction.csv"
df_filtrado.to_csv(file_name)

In [39]:
file_name = "df_58_for_prediction.csv"
df_58.to_csv(file_name)

In [40]:
file_name = "df_calendar_clean.csv"
df_calendar_with_events.to_csv(file_name)